In [10]:
from tf_agents.environments import suite_gym

env = suite_gym.load("SpaceInvaders-v4")
env.step(1).observation.shape

(210, 160, 3)

In [4]:
env.gym.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [7]:
from tf_agents.environments import suite_atari
from tf_agents.environments.atari_preprocessing import AtariPreprocessing
from tf_agents.environments.atari_wrappers import FrameStack4

max_episode_steps = 27000
environment_name = "SpaceInvadersNoFrameskip-v4"

# ?
# class AtariPreprocessingWithAutoFire(AtariPreprocessing):
#     def reset(self, **kwargs):
#         obs = super().reset(**kwargs)
#         super().step(1) # FIRE to start
#         return obs
#     def step(self, action):
#         lives_before_action = self.ale.lives()
#         obs, rewards, done, info = super().step(action)
#         if self.ale.lives() < lives_before_action and not done:
#             super().step(1) # FIRE to start after life lost
#         return obs, rewards, done, info

env = suite_atari.load(
    environment_name,
    max_episode_steps=max_episode_steps,
    gym_env_wrappers=[AtariPreprocessing, FrameStack4])
env

In [8]:
env.seed(42)
env.reset()
for _ in range(4):
    time_step = env.step(3) # LEFT
    print(time_step.observation.shape)

(84, 84, 4)
(84, 84, 4)
(84, 84, 4)
(84, 84, 4)
